In [1]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader  

# Check that MPS is available

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )
    mps_device = None
else:
    mps_device = torch.device("mps")

if mps_device is not None:
    device = mps_device
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

device

MPS not available because the current PyTorch install was not built with MPS enabled.
Using GPU: NVIDIA GeForce RTX 4080 Laptop GPU


device(type='cuda')

In [2]:
%%time

# Define the dataset class
class RatingDataset(Dataset):
    """Dataset for loading user-item ratings for training"""
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = torch.tensor(user_ids, dtype=torch.int64)
        self.item_ids = torch.tensor(item_ids, dtype=torch.int64)
        self.ratings = torch.tensor(ratings, dtype=torch.float32)

    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

class WideAndDeep(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20, hidden_layers=[64, 32, 16], dropout_rate=0.2):
        super(WideAndDeep, self).__init__()
        # Wide part
        self.wide = nn.Embedding(num_users + num_items, 1)
        # Deep part
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc_layers = nn.ModuleList()
        input_size = 2 * embedding_dim  # Concatenated user and item embeddings
        for hidden_layer in hidden_layers:
            self.fc_layers.append(nn.Linear(input_size, hidden_layer))
            input_size = hidden_layer
        self.fc_layers.append(nn.Linear(input_size, 1))  # Final output layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, user_indices, item_indices):
        # Wide part
        wide_out = self.wide(user_indices + item_indices)
        # Deep part
        user_embedding = self.user_embedding(user_indices)
        item_embedding = self.item_embedding(item_indices)
        x = torch.cat([user_embedding, item_embedding], dim=-1)
        for layer in self.fc_layers[:-1]:
            x = self.relu(layer(x))
            x = self.dropout(x)
        deep_out = self.fc_layers[-1](x)
        # Combined part
        combined_out = wide_out + deep_out
        return combined_out.squeeze()

# Training function
def train_model(model, data_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (users, items, ratings) in enumerate(data_loader):
            users = users.to(device)  # Move data to GPU
            items = items.to(device)  # Move data to GPU
            ratings = ratings.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(users, items)
            loss = criterion(outputs, ratings)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if batch_idx % 100 == 99:
                print(f'Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

# Prepare data
train_data = pd.read_csv("cs608_ip_train_v3.csv")
train_data['user_id'] = train_data['user_id'].astype('category').cat.codes
train_data['item_id'] = train_data['item_id'].astype('category').cat.codes
dataset = RatingDataset(train_data['user_id'], train_data['item_id'], train_data['rating'])

# Create DataLoader
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize model, loss, and optimizer
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
model = WideAndDeep(num_users, num_items).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, data_loader, criterion, optimizer, epochs=10)

Epoch 1, Batch 100, Loss: 8.9168
Epoch 1, Batch 200, Loss: 3.5820
Epoch 1, Batch 300, Loss: 3.5746
Epoch 1, Batch 400, Loss: 3.3771
Epoch 1, Batch 500, Loss: 3.3333
Epoch 1, Batch 600, Loss: 3.3555
Epoch 1, Batch 700, Loss: 3.2803
Epoch 1, Batch 800, Loss: 3.1857
Epoch 1, Batch 900, Loss: 3.1720
Epoch 1, Batch 1000, Loss: 3.0940
Epoch 1, Batch 1100, Loss: 3.0077
Epoch 1, Batch 1200, Loss: 3.0645
Epoch 1, Batch 1300, Loss: 3.0632
Epoch 1, Batch 1400, Loss: 2.9743
Epoch 1, Batch 1500, Loss: 2.7887
Epoch 1, Batch 1600, Loss: 2.9168
Epoch 1, Batch 1700, Loss: 2.8890
Epoch 1, Batch 1800, Loss: 2.8450
Epoch 1, Batch 1900, Loss: 2.9801
Epoch 1, Batch 2000, Loss: 2.6914
Epoch 1, Batch 2100, Loss: 2.7990
Epoch 1, Batch 2200, Loss: 2.7569
Epoch 1, Batch 2300, Loss: 2.7950
Epoch 1, Batch 2400, Loss: 2.7273
Epoch 1, Batch 2500, Loss: 2.6596
Epoch 1, Batch 2600, Loss: 2.6867
Epoch 1, Batch 2700, Loss: 2.7836
Epoch 1, Batch 2800, Loss: 2.7150
Epoch 1, Batch 2900, Loss: 2.6132
Epoch 2, Batch 100, Los

In [3]:
# Save the model
torch.save(model.state_dict(), "wide_and_deep.pth")

In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, accuracy_score

# Load your CSV data
data_path = "./cs608_ip_probe_v3.csv"
df = pd.read_csv(data_path)

# Assume df has columns 'user_id', 'item_id', which we need to convert to tensor
# Also assume that 'ratings' column is your target
users = torch.tensor(df["user_id"].values).to(device)
items = torch.tensor(df["item_id"].values).to(device)
ratings = torch.tensor(df["rating"].values)

model.eval()  # Set the model to evaluation mode

# Create a data loader for batch processing
dataset = TensorDataset(users, items, ratings)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False)

# To store predictions and actual values
predictions, actuals = [], []

# Evaluate the model
for user, item, rating in data_loader:
    with torch.no_grad():
        output = model(user, item)
        predictions.extend(output.cpu().numpy())
        actuals.extend(rating.cpu().numpy())

# Calculate evaluation metrics
predictions = np.array(predictions)
actuals = np.array(actuals)
rmse = np.sqrt(mean_squared_error(actuals, predictions))
accuracy = accuracy_score(actuals, predictions.round())

print(f"RMSE: {rmse}")
print(f"Accuracy: {accuracy}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from tqdm.notebook import tqdm

def generate_recommendations(model, num_users, num_items, top_k=50):
    model.eval()  # Set the model to evaluation mode
    recommendations = []

    # Iterate over all users
    for user_id in tqdm(range(num_users)):
        user_tensor = torch.tensor(
            [user_id] * num_items, dtype=torch.int64
        ).to(device)  # Repeat user ID for each item
        item_tensor = torch.tensor(range(num_items), dtype=torch.int64).to(device)  # All item IDs

        # Predict scores for all items for this user
        with torch.no_grad():
            scores = (
                model(user_tensor, item_tensor).cpu().numpy()
            )  # Get scores and move to CPU

        # Get the indices of the top k scores
        top_item_indices = scores.argsort()[-top_k:][
            ::-1
        ]  # Indices of top scoring items

        # Append to the list of recommendations
        recommendations.append(top_item_indices.tolist())

    return recommendations

In [ ]:
%%time

import zipfile

# Number of users and items
num_users = train_data["user_id"].nunique()
num_items = train_data["item_id"].nunique()

# Generate recommendations for all users
top_k_recommendations = generate_recommendations(model, num_users, num_items)

with open("submission.txt", "w") as file:
    for user_recommendations in top_k_recommendations:
        file.write(" ".join(map(str, user_recommendations)) + "\n")

# zip the submission file
with zipfile.ZipFile('submission.zip', 'w') as file:
    file.write('submission.txt')

  0%|          | 0/21124 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 